<a href="https://colab.research.google.com/github/JJnotJimmyJohn/CodeAlongNLPwithPyTorch/blob/master/example_PyTorchEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Toy Embedding

In [64]:
embeds = nn.Embedding(2,5)

In [65]:
embeds(torch.tensor(1,dtype=torch.long))

tensor([-0.2899, -0.3460,  0.7215, -0.4418,  0.6033],
       grad_fn=<EmbeddingBackward>)

# N-gram Language modeling

In [66]:
from collections import namedtuple

In [67]:
Config = namedtuple('Config',['context_size','embedding_dim'])

In [68]:
args = Config(2,10)

In [69]:
args.context_size

2

In [70]:
args.embedding_dim

10

In [71]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [72]:
ngrams = [(test_sentence[i:i+args.context_size],test_sentence[i+args.context_size]) for i in range(len(test_sentence)-args.context_size)]
ngrams[:3]

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege')]

In [73]:
# assign each word a index, later we will use that index to locate the embedding
words = list(set(test_sentence))
vocab_size = len(words)
word_to_idx = {word:num for word, num in zip(words,range(len(words)))}

In [74]:
print(word_to_idx)

{'cold.': 0, 'much': 1, 'gazed': 2, 'Then': 3, 'the': 4, 'my': 5, 'more': 6, 'thine': 7, 'days;': 8, 'Were': 9, 'to': 10, 'Where': 11, 'and': 12, 'child': 13, 'trenches': 14, "beauty's": 15, 'shame,': 16, 'Thy': 17, 'dig': 18, "'This": 19, 'praise.': 20, 'all-eating': 21, 'treasure': 22, 'Proving': 23, 'This': 24, 'succession': 25, 'livery': 26, 'an': 27, "excuse,'": 28, 'see': 29, 'were': 30, "feel'st": 31, 'when': 32, 'proud': 33, 'besiege': 34, 'Shall': 35, 'held:': 36, 'weed': 37, 'be': 38, 'eyes,': 39, 'art': 40, 'all': 41, "deserv'd": 42, 'old': 43, 'new': 44, 'mine': 45, 'it': 46, 'couldst': 47, 'answer': 48, 'now,': 49, 'thou': 50, 'his': 51, 'shall': 52, 'forty': 53, 'praise': 54, "youth's": 55, 'so': 56, 'small': 57, 'use,': 58, 'warm': 59, 'made': 60, 'within': 61, 'own': 62, 'worth': 63, 'Will': 64, 'of': 65, 'fair': 66, 'How': 67, 'deep': 68, 'thriftless': 69, 'blood': 70, 'on': 71, 'thy': 72, 'old,': 73, 'by': 74, 'And': 75, 'sum': 76, 'being': 77, 'lies,': 78, 'count,': 

# Model

In [75]:
class NGramLM(nn.Module):
  def __init__(self,vocab_size,embedding_dim,context_size):
    super(NGramLM,self).__init__()
    # first get an embedding layer
    self.embeddings = nn.Embedding(vocab_size,embedding_dim)
    # input size = context words size * embedding dimension
    self.linear1 = nn.Linear(embedding_dim*context_size,128)
    # input size = last output size
    # need vocab_size vector to guess which word that is
    self.linear2 = nn.Linear(128,vocab_size)

  def forward(self,inputs):
    embeds = self.embeddings(inputs).view((1,-1))
    out = F.relu(self.linear1(embeds))
    out = self.linear2(out)
    log_probs = F.log_softmax(out,dim=1)
    return log_probs

In [76]:
# set up loss function and optimizer
losses = []
loss_function = nn.NLLLoss()

model = NGramLM(vocab_size,args.embedding_dim,args.context_size)
optimizer = optim.SGD(model.parameters(),lr=0.001)

In [77]:
model.embeddings(torch.tensor(2,dtype=torch.long))

tensor([-1.8326,  0.3169,  0.4066,  0.9902,  0.6498,  0.5291, -0.8644,  0.3723,
        -0.3940,  0.0707], grad_fn=<EmbeddingBackward>)

In [85]:
for epoch in range(10):
  total_loss = 0
  for context, target in ngrams:

    embed_idxs = torch.tensor([word_to_idx[word] for word in context],dtype=torch.long)
    model.zero_grad()
    
    log_probs = model(embed_idxs)

    loss = loss_function(log_probs,torch.tensor([word_to_idx[target]], dtype=torch.long))

    # backward and update parameters
    loss.backward()
    optimizer.step()

    total_loss+=loss.item()
  losses.append(total_loss)
print(losses)

[519.6402027606964, 517.3343765735626, 515.0431771278381, 512.7640767097473, 510.49564933776855, 508.2377498149872, 505.98920369148254, 503.75186586380005, 501.52382731437683, 499.30370926856995]


In [86]:
print(1)

1
